## Fine tunning Llama 3.2 B model

We will use unsloth library

You're fine-tuning your model to become better at instruction-following, multi-domain generalization, and chat-style completion by using FineTome-100k. Think of this as making your model closer to ChatGPT-style assistants by supervised fine-tuning on high-quality, curated tasks.

We are doing instruction fine-tuning using a LoRA-adapted quantized language model on the FineTome-100k dataset, with the goal of:

1) Making the model better at responding to chat-style inputs

2) Teaching it to follow instructions

3) Aligning it with human-like reasoning and answers



In [1]:
 pip install unsloth transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.0/277.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/

In [2]:
import torch # For using neural networks updation and taking care of weights
from unsloth import FastLanguageModel# For loading the pretrained model and for peft
from datasets import load_dataset # To load the datasets
from trl import SFTTrainer# For making trainer and using for Supervised fine tunning
from transformers import TrainingArguments # For giving arguments in the training of trainer
from unsloth.chat_templates import get_chat_template,standardize_sharegpt## create templates to make so that it is used for training model

In [ ]:
model,tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",# Load Pretrained Model and Enables LORA AND QLORA FOR adaptation
    max_seq_length=2048,# Maxm number ofgit  token that a model can handle
    load_in_4bit=True,# Quantized the model into 4 bit model
)

==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


In [15]:
model=FastLanguageModel.get_peft_model(
    model,r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)

In [12]:
tokenizer=get_chat_template(tokenizer,chat_template="llama-3.1")
## It give tokenizer to be make conversation between user and assistant into a single
# string as in the format of Llama-3.1 model

In [8]:
dataset=load_dataset("mlabonne/FineTome-100k",split="train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [9]:
dataset=standardize_sharegpt(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [10]:
dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [13]:
dataset=dataset.map(
    lambda examples:{
        "text":[
            tokenizer.apply_chat_template(convo,tokenize=False)
            for convo in examples["conversations"]
        ]
    },
    batched=True,
)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [24]:
trainer=SFTTrainer(# we are creating a object from SFTTrainer class from trl library
    model=model ,## provide peft model for fine tunning
    train_dataset= dataset ,# in which dataset u have to fine tune it in which there is a text field
    dataset_text_field="text",# column in which u have to fine tune your model present in your dataset
    max_seq_length=1024,# Maximum input token for input
    args=TrainingArguments(## Training hyperparameters
        per_device_train_batch_size=1,# number of samples per GPU per batch
        gradient_accumulation_steps=4,# Accumulate gradient step before updating it
        warmup_steps=5,## slowly ramps to increaes learning rate
        max_steps=60,#Total training steps
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(), #choose between what is present
        logging_steps=1,# to log about the loss in every one step
        output_dir="outputs" # Path to save checkpoints like final_model and logs
    ),
)

In [25]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed expli

Unsloth: Will smartly offload gradients to save VRAM!


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

Step,Training Loss
1,1.323500
2,1.581600
3,1.704900
4,1.910900
5,1.291900
6,1.175400
7,1.235300
8,1.226700
9,1.116500
10,1.227800


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=60, training_loss=1.0516178588072458, metrics={'train_runtime': 205.6823, 'train_samples_per_second': 1.167, 'train_steps_per_second': 0.292, 'total_flos': 2188400906668032.0, 'train_loss': 1.0516178588072458})

In [27]:
model.save_pretrained("final_model")## To save the model

In [28]:
inference_model,inference_tokenizer=FastLanguageModel.from_pretrained(
    model_name="final_model",
    max_seq_length=1024,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


In [32]:
text_prompts = [
    "What are the key principles of investment?"
]

for prompt in text_prompts:
  formatted_prompt = inference_tokenizer.apply_chat_template([{
      "role": "user",
      "content": prompt
      }], tokenize=False)

  model_inputs = inference_tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
  generated_ids = inference_model.generate(
      **model_inputs,
      max_new_tokens=512,
      temperature=0.7,
      do_sample=True,
      pad_token_id=inference_tokenizer.pad_token_id
  )
  response = inference_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  print(response)

system

Cutting Knowledge Date: December 2023
Today Date: 20 Jun 2025

user

What are the key principles of investment?assistant

The key principles of investment include:

1. Diversification: Spreading investments across various asset classes to minimize risk and maximize returns.
2. Risk management: Understanding and managing risk to achieve investment objectives.
3. Long-term perspective: Investing for the long-term, rather than focusing on short-term gains.
4. Dollar-cost averaging: Investing a fixed amount of money at regular intervals, regardless of market conditions.
5. Diversification of income: Investing in a mix of income-generating assets to reduce reliance on any one source.
6. Active management: Regularly monitoring and adjusting investments to respond to market conditions.
7. Tax efficiency: Considering tax implications when making investment decisions.
8. Liquidity: Ensuring that investments can be easily converted into cash when needed.
9. Risk assessment: Continuously 